<a href="https://colab.research.google.com/github/jeremiahoclark/open_source_colabs/blob/main/download_linkedin_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,349 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,843 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricte

In [2]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from google.colab import files

import pandas as pd
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# # set the target URL
# url = "put-url-here-to-scrape"

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)


In [3]:
profile_link = input("Enter the LinkedIn profile link: ")
linkedin_at_cookie = input("Enter the linkedin_at cookie value: ")
# Set the cookie in the webdriver
driver.get('https://www.linkedin.com')
driver.add_cookie({'name': 'linkedin_at', 'value': linkedin_at_cookie})

Enter the LinkedIn profile link: https://www.linkedin.com/in/jadebonacolta/
Enter the linkedin_at cookie value: AQEDARP6tUwCGEcpAAABjEAEevMAAAGOQpe8Ok0AvjDPdrWRMpFH6hsx1QenkIGcX1QyJfPSPMJKY4Zfi5IGeo209MlZoOYkOC3NMpIoNSshC_C26O7YQhrKMl2dIV-bHhISvzx8tTTBO4ahjbTH4fWX


In [4]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Set the cookie in the webdriver
driver.get('https://www.linkedin.com')
driver.add_cookie({'name': 'linkedin_at', 'value': linkedin_at_cookie})

# Navigate to the activities page
url = profile_link
if not url.endswith('/recent-activity/'):
    url += '/recent-activity/'
driver.get(url)
print("Navigated to:", url)

try:
    # Wait for the activities to be present
    activities = WebDriverWait(driver, 10).until(
        lambda x: x.find_elements(By.CSS_SELECTOR, '.occludable-update')
    )

    activity_list = []
    for activity in activities:
        activity_text = activity.text
        print("Activity text:", activity_text)
        activity_dict = {'text': activity_text}
        activity_list.append(activity_dict)

    # Save as JSON file
    with open('activities.json', 'w') as f:
        json.dump(activity_list, f)
    print("Saved activities to activities.json")

    # Download the JSON file
    files.download('activities.json')
    print("Downloaded activities.json")

except TimeoutException:
    print("Timed out waiting for activities to load.")

except NoSuchElementException:
    print("Activities not found on the page.")

finally:
    # Close the webdriver
    driver.quit()

Navigated to: https://www.linkedin.com/in/jadebonacolta//recent-activity/
Timed out waiting for activities to load.


Archive